In [0]:
import pandas as pd
import numpy as np
import cv2
import pickle

In [0]:
from time import time
import keras
import keras.backend as K
from keras import metrics
from keras.engine.topology import Layer, InputSpec
from keras.layers import Dense, Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from keras.models import Model
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Concatenate, Flatten, Reshape
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
def load_file(nome_arquivo):
    with open(nome_arquivo, 'rb') as input:
        objeto = pickle.load(input)
    return objeto

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
comandos = pd.concat([pd.DataFrame(load_file('/content/drive/My Drive/Colab Notebooks/comando.pkl')),\
          pd.DataFrame(load_file('/content/drive/My Drive/Colab Notebooks/comando2.pkl'))],\
          axis=0)

In [0]:
comandos.describe()

,0,1,2,3
count,2612.000000,2612.000000,2612.000000,2612.000000
mean,0.001531,0.018377,0.795559,0.017228
std,0.660186,0.145292,0.403370,0.130146
min,-1.000000,-1.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000
75%,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [0]:
comandos['direita'] = comandos[0].map(lambda x: 1 if x == 1 else 0)
comandos['esquerda'] = comandos[0].map(lambda x: 1 if x == -1 else 0)
comandos['cima'] = comandos[1].map(lambda x: 1 if x == 1 else 0)
comandos['baixo'] = comandos[1].map(lambda x: 1 if x == -1 else 0)

In [0]:
comandos = comandos[[2,3,'direita','esquerda','cima','baixo']].rename(index=str, columns={2:'acelera',3:'freia'})

In [0]:
imagens = []
for img in load_file('/content/drive/My Drive/Colab Notebooks/img.pkl'):
  imagens.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)/255)
for img in load_file('/content/drive/My Drive/Colab Notebooks/img2.pkl'):
  imagens.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)/255)

In [0]:
#teste
#gray = cv2.cvtColor(imagens[0], cv2.COLOR_BGR2GRAY)
#cv2.imwrite('/content/drive/My Drive/Colab Notebooks/img.jpg',gray)

In [0]:
imagens =  np.array(imagens).reshape((2612, 220, 320,1))

In [0]:
#input_shape = (img_rows, img_cols, 1)
input_shape = (220, 320,1)
num_classes = comandos.shape[1]
batch_size = 64
epochs = 100

In [0]:
# CRIANDO MODELO

"""
model = Sequential()
model.add(Conv2D(128, kernel_size=(8, 8),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))
"""
model = Sequential()
model.add(Conv2D(24, kernel_size=(5, 5), strides=(2, 2), activation='relu', input_shape=input_shape))
model.add(Conv2D(36, kernel_size=(5, 5), strides=(2, 2), activation='relu'))
model.add(Conv2D(48, kernel_size=(5, 5), strides=(2, 2), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(2056, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 108, 158, 24)      624       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 52, 77, 36)        21636     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 37, 48)        43248     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 22, 35, 64)        27712     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 20, 33, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 16, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 16, 64)        0         
__________

In [0]:
imagens.shape[0]

2612

In [0]:
# train, test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(range(0,imagens.shape[0]),range(0,imagens.shape[0]), test_size=0.05, random_state=42)

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

W0731 00:18:54.554337 140388338050944 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0731 00:18:54.563902 140388338050944 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
imagens[X_train,:,:,:].shape

(2481, 220, 320, 1)

In [0]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=20,verbose=0, mode='min')
mcp_save = ModelCheckpoint('/content/drive/My Drive/Colab Notebooks/modelo_outrun_2.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

model.fit(imagens[X_train,:,:,:], comandos.iloc[X_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(imagens[X_test,:,:,:], comandos.iloc[X_test]),
          callbacks=[earlyStopping,reduce_lr_loss,mcp_save])
score = model.evaluate(imagens[X_test,:,:,:], comandos.iloc[X_test], verbose=0)
print('Test loss:', score[0])

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '
W0731 00:19:13.867279 140388338050944 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2481 samples, validate on 131 samples
Epoch 1/100
2481/2481 [==============================] - 12s 5ms/step - loss: 1.7488 - acc: 0.8138 - val_loss: 1.2948 - val_acc: 0.9237
Epoch 2/100
2481/2481 [==============================] - 3s 1ms/step - loss: 1.5398 - acc: 0.8928 - val_loss: 1.2724 - val_acc: 0.9237
Epoch 3/100
2481/2481 [==============================] - 3s 1ms/step - loss: 1.4981 - acc: 0.8996 - val_loss: 1.2094 - val_acc: 0.9237
Epoch 4/100
2481/2481 [==============================] - 3s 1ms/step - loss: 1.4629 - acc: 0.9093 - val_loss: 1.2027 - val_acc: 0.9237
Epoch 5/100
2481/2481 [==============================] - 3s 1ms/step - loss: 1.4307 - acc: 0.9125 - val_loss: 1.1916 - val_acc: 0.9237
Epoch 6/100
2481/2481 [==============================] - 3s 1ms/step - loss: 1.4209 - acc: 0.9113 - val_loss: 1.1846 - val_acc: 0.9237
Epoch 7/100
2481/2481 [==============================] - 3s 1ms/step - loss: 1.4240 - acc: 0.9141 - val_loss: 1.1930 - val_acc: 0.9237
Epoch 8

In [0]:
resultados = pd.DataFrame(model.predict_proba(imagens),columns=['acelera','freia','direita','esquerda','cima','baixo']).round(2)

In [0]:
resultados.describe()

,acelera,freia,direita,esquerda,cima,baixo
count,2612.000000,2612.000000,2612.000000,2612.000000,2612.000000,2612.000000
mean,0.978185,0.030965,0.283488,0.274747,0.023224,0.005310
std,0.085541,0.117292,0.424067,0.417099,0.083751,0.023454
min,0.430000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.730000,0.770000,0.000000,0.000000
max,1.000000,0.850000,1.000000,1.000000,0.510000,0.180000
